## Elixer Widget

In order to streamline classifications we have developed a GUI using the ipywidgets package to quickly scan through ELixer report summaries and store source classifications. If you do not have previous experience classifying with these reports, we highly recommend you read the Elixer_readme.pdf located in 
```
https://github.com/HETDEX/elixer/blob/hdr2/docs/Elixer_readme.pdf
```
ELiXer reports allow us to visually classify HETDEX line emitting galaxies by combining HETDEX fiber spectra and 2D CCD images of line detections with ancillary photometric images and catalogs. You only need to know the detectid of a source to look up the ELiXer report.

You can either use the widget to explore a subset of sources from the detections database
The GUI will take either a detectlist in the form of a numpy array, a saved detectlist (saved previously using np.savetxt), or a saved classification table file. You can only classify objects within this input list. You may also specify the name of the output file for the classification table.

### Launch the widget

The widget is launched by calling the class `ElixerWidget()` from `hetdex_api.elixer_widget_cls`. When you launch the widget you are generating a dictionary to store your visual classifications as well as initializing the widget. It either loads in a pre-defined detection list or if no argument is given it loads a list of all HDR2 detections.

First run this to prevent scrolling in individual cells in the jupyter notebook

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np

In [3]:
from hetdex_api.elixer_widget_cls import ElixerWidget
from hetdex_api.detections import Detections

### Get Elixers for a detections database query

`Detections()` initiates the detections class and stores each array from the HDF5 detections table as array attributes that can easily be indexed through numpy. It also populates attribute arrays with ELiXeR OII to LAE probabilities (note these are preliminary) as well as measuring an approximate gband magnitude from the 1D HETDEX spectra. If you call it with the `refine()` method option you will automatically have all bad detections removed from the database (for example, newly discovered bad amps, bad detectids due software issues or HDR2 shots that we have now decided should not be used for scientific analysis for various reasons.) You may also optionally add a refine(gmagcut=XX) option to remove all sources brighter than that value.

In [4]:
detects = Detections('hdr2').refine(gmagcut=23)

For example here is a selection that would help us find a sample of nice bright LAEs, and of course some interloping OII emitters:

In [5]:
sel = (detects.gmag > 23.5) * (detects.sn > 10) * (detects.date >= 20190101) 
np.sum(sel)

6592

Then insert the selected detectid list to the `detectlist` option in the ElixerWidget call. You can click up and down your list using the arrows. The neighbors button is a great new feature that will allow you to look into all nearby sources to the detection. If there is a catalog match, you can grab the spectrum quickly to help you decide which catalog match best matces the detection.

In [7]:
elix_widget = ElixerWidget(detectlist=detects.detectid[sel]) 

interactive(children=(BoundedIntText(value=2000452673, description='DetectID:', max=9900000000, min=1000000000…

### Open up a file with a list of detectids

To open the GUI to explore the previous detectid list:

In [7]:
np.savetxt('detectLAEs.txt', detects.detectid[sel], fmt='%i')

In [8]:
elix_widget = ElixerWidget(detectfile='detectLAEs.txt')

interactive(children=(BoundedIntText(value=2000452673, description='DetectID:', max=9900000000, min=1000000000…

Or if you have done some classifications previously and want to continue with the list (by default this list is saved as `elixer_classifications.dat`). If you use the resume=True flag, the detectID will start after the highest detectID classified previously. 

In [32]:
# elix_widget = ElixerWidget(savedfile='elixer_classifications.dat', resume=True)

### Where the classifications are stored

`elix_widget` is a class object that contains arrays of the input detectid, vis_class and comment values output from the above GUI. It can be saved and shared with the group using the `Save Progress` button above. It saves

In [38]:
elix_widget.detectid

array([2000452673, 2000452715, 2000452896, ..., 2001095016, 2001095017,
       2001095018])

In [39]:
elix_widget.vis_class

array([5, 5, 0, ..., 0, 0, 0])

The output is ingested into an astropy Table object under the attribute `output` and then saved to a txt file. The file name will be the savedfile name if provided to the widget upon intialization. Or you can provide the file name as outfile='filename.dat' as an argument upon initialization. By default, it will save the table to `elixer_cls.dat`

In [41]:
elix_widget.output

detectid,vis_class,flag,z,counterpart,comments
int64,int64,int64,float64,int64,bytes80
2000452673,5,1,-1.0,-1,?
2000452715,5,1,-1.0,-1,?
2000452896,0,0,-1.0,-1,?
2000452913,0,0,-1.0,-1,?
2000453037,0,0,-1.0,-1,?
2000453044,0,0,-1.0,-1,?
2000453175,0,0,-1.0,-1,?
2000453197,0,0,-1.0,-1,?
2000453204,0,0,-1.0,-1,?


### Classifying a pre-defined list

Most often, we will provide the team with a pre-made list to classify. Use the `savedfile` and `resume=True` options so that your classifiying list resumes where you last left off.

In [44]:
from hetdex_api.elixer_widget_cls import ElixerWidget
from hetdex_api.detections import Detections

In [48]:
elix_widget = ElixerWidget(savedfile='elixer_hdr2_testlist.dat', resume=True)

interactive(children=(BoundedIntText(value=2000452715, description='DetectID:', max=9900000000, min=1000000000…